# Práctica: Imputación de datos ausentes

Usaremos la librería `VIM` para la imputación de datos ausentes. En esta práctica:

* Nos apoyaremos en la inspección realizada en la práctica de visualización de datos ausentes para imputar ausencias en el conjunto de datos jovenes.rdata.
* Aprenderemos a:

    * Imputar por los métodos: media, media condicionada, hot-deck secuencial, hot-deck aleatorio y la imputación basada en knn.
    * Comparar visualmente los efectos de la imputación.


In [ ]:
# install.packages('VIM')
library(VIM)
library(naniar)  # para la visualización de ausencias
library(ggplot2) # para hacer gráficas con el efecto de la imputación

## Los datos: respuestas cuestionario jóvenes

Para realizar esta práctica se usará el fichero de datos `jovenes.rdata`, que contiene las respuestas de más de 600 jóvenes de Gijón al cuestionario `cuestionario.pdf`.

In [ ]:
load("jovenes.rdata")  # carga los datos en formato .rdata
ls()                   # para comprobar el nombre del objeto cargado 

head(data)
dim(data) # dimensión del dataframe

Vamos a ver qué variables podemos imputar en esta práctica, es decir, qué variables son numéricas y tienen ausencias: 

In [ ]:
options(repr.plot.width=12, repr.plot.height=5)
????::?????(data)

En esta práctica imputaremos dos variables: P12 (*horas semanales de ocio*) y P16 (*edad de la primera relación sexual con penetración*), pues en la práctica anterior observamos que había diferencias en las ausencias de esta última entre los encuestados de sexo masculino y femenino.

## Búsqueda de datos erróneos

Antes de imputar, debemos preguntarnos si existen datos erróneos en esta variable. Podemos verlo de distintas maneras: histograma, resumen, etc.

In [ ]:
??????

En caso de encontrar algún valor erróneo, habría que transformarlo en NA e imputarlo después. La variable P12 contiene algunos valores que podrían considerarse erróneos: si suponemos que el número máximo de horas semanales que una persona puede dedicar al ocio es de 84 (12 horas diarias) nos encontramos con que hay 15 encuestados que superan ese umbral.

In [ ]:
# comprueba cuántos encuestados lo cumplen

Hay que sustituir esos valores por NAs en P12, pero antes hay equ guardar esta información en otra variable para "guardar la traza de los datos". Creamos una variable informativa que tome el valor 0 si P12 es "correcta", 1 si es ausente y 2 si es un error (esto puede hacerse de distintas formas).

In [ ]:
data$P12_cod <-  ????

# sustituimos los valores erróneos 
data$P12 <- ????


## Imputación por la media

En todos los métodos procederemos de la misma forma:

* Paso 1. Cálculo de la media y desviación típica antes de imputar.
* Paso 2. Imputación.
* Paso 3. Cálculo de la media y desviación típica después de imputar.
* Paso 4. Comparación de resultados gráficamente.

In [ ]:
# 1). Calculamos media y desviación típica para ver cómo cambian al imputar
m_P12 <- ????
sigma_P12 <- ????
m_P16 <- ????
sigma_P16 <- ????

# 2). Imputamos
data$P12_imp <- ????
data$P16_imp <- ????

# 3). media y desviación típica tras imputar
m_P12_imp <- ????
sigma_P12_imp <- ????
m_P16_imp <- ????
sigma_P16_imp <- ????

# 4). Pintamos
titulo_P12 <- paste0("IMPUTACIÓN POR LA MEDIA NO CONDICIONADA\nAntes de imputar: media ", round(m_P12,2),
                     "\t  -  Std: ", round(sigma_P12,2),
                     "\nDespués de imputar: media ", round(m_P12_imp, 2),
                     "\t  -  Std: ", round(sigma_P12_imp,2))
titulo_P16 <- paste0("IMPUTACIÓN POR LA MEDIA NO CONDICIONADA\nAntes de imputar: media ", round(m_P16,2),
                     "\t  -  Std: ", round(sigma_P16,2),
                     "\nDespués de imputar: media ", round(m_P16_imp, 2),
                     "\t  -  Std: ", round(sigma_P16_imp,2))

plot_P12 <- ggplot(data) + 
              geom_density(aes(x=P12), size = 1, colour = "black") +
              geom_density(aes(x=P12_imp), size = 1, colour = "red") + 
              ggtitle(titulo_P12) + theme_minimal()

plot_P16 <- ggplot(data) + 
              geom_density(aes(x=P16), size = 1, colour = "black") +
              geom_density(aes(x=P16_imp), size = 1, colour = "red") + 
              ggtitle(titulo_P16) + theme_minimal()

plot_P12
plot_P16

**BONUS**: si quieres ver dos gráficos de `ggplot2` en un mismo "lienzo", puedes usar la librería `patchwork`: 

In [ ]:
#install.packages('patchwork')
library(patchwork)
plot_P12 + plot_P16  # una gozada de sintaxis :D

## Imputación por media condicionada

Nos basaremos en la variable P2 (*sexo*) para crear grupos en la muestra sobre los que calcular la media condicionada. Antes de proceder, eliminaremos las ausencias de esta variable:

In [ ]:
data <- ????

Hay múltiples formas de calcular la media de una o varias variables por grupos. Una opción interesante es la que ofrece la función `summaryBy` de la librería `doBy`:

In [ ]:
library(doBy)

# Calculamos media condicionada
mean_na <- ????
df <- ????
df

In [ ]:
# 2). Imputamos
data$P12_imp <- ifelse(is.na(data$P12) & data$P2 == 'Femenino', df$P12.mean_na[1], data$P12)
data$P12_imp <- ifelse(is.na(data$P12) & data$P2 == 'Masculino', df$P12.mean_na[2], data$P12)
data$P16_imp <- ifelse(is.na(data$P16) & data$P2 == 'Femenino', df$P16.mean_na[1], data$P16)
data$P16_imp <- ifelse(is.na(data$P16) & data$P2 == 'Masculino', df$P16.mean_na[2], data$P16)

# copia y pega los pasos 3 y 4, modificando el título del gráfico :D

## Imputación por hot-deck secuencial

Nos basaremos en la variable P1 (*identificador*) para ordenar las filas de la muestra y poder imputar con el método Hot-Deck secuencial:

In [ ]:
# 2). Imputamos
data_imputed <- ???? # esta función devuelve el dataset imputado

# copia y pega los pasos 3 y 4, modificando el título del gráfico :D

## Imputación por hot-deck aleatorio

Igual que el caso anterior, pero sin usar el argumento `ord_var`.

**Nota**: si queremos, podemos condicionar la elección por grupos. Para ello, basta con usar el argumento `domain_var` e indicar cuál es la variable (o variables) que forman los grupos. 

In [ ]:
# 2). Imputamos
data_imputed <- ???? # esta función devuelve el dataset imputado

# copia y pega los pasos 3 y 4, modificando el título del gráfico :D

## Imputación basada en knn

In [ ]:
# 2). Imputamos
data_imputed <- ????? # esta función devuelve el dataset imputado
# es una función flexible - mucha configuración si se desea

# copia y pega los pasos 3 y 4, modificando el título del gráfico :D